In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator
import sys
%matplotlib inline

In [2]:
plt.rcParams["font.family"] = 'NanumGothic'

In [3]:
df = pd.read_csv('../input/funda_train.csv')

In [4]:
df.head(5)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [5]:
df['store_id'] = df['store_id'].astype(np.int16)
df['installment_term'] = df['installment_term'].astype(np.int16)

In [6]:
def split_data(x, idx, seg):
    if type(x) is str:
        return x.split(seg)[idx]
    return None

## transacted_year

In [7]:
df['transacted_year'] = df['transacted_date'].apply(split_data, args=[0, '-'])
df['transacted_month'] = df['transacted_date'].apply(split_data, args=[1, '-'])

In [8]:
df['transacted_hour'] = df['transacted_time'].apply(split_data, args=[0, ':'])
df['transacted_hour'] = df['transacted_hour'].astype(np.int16)

## type_of_business

In [9]:
df['type_big'] = df['type_of_business'].apply(split_data, args=[-1, ' '])

## region

In [10]:
def check_store_region(store_id):
    #해당 가게 데이터를 가져온다
    store = df[df['store_id']==store_id][['region', 'card_id']]
    region_dict = {}
    #지역이 None이면
    if sum(store['region'].isnull()):
        #해당 가게에서 사용되 card_id를 확인
        store_card_id = store['card_id'].unique()
        #각 card_id에서
        for card_id in store_card_id:
            #해당 card_id가 사용된 지역들을 찾는다.
            regions = df[df['card_id']==card_id]['region'].unique()
            #각 지역을 region_dict에 추가한다
            for region in regions:
                if region not in region_dict:
                    region_dict[region] = 1
                else:
                    region_dict[region] += 1
    region_dict.pop(np.nan, None)
    return region_dict

In [11]:
#같은 값을 가지는 값이 있을 수 있다.
#대도시를 중심으로 대도시의 수에 따라 각 값에 가중치를 더한다.
#부산 금정구:1, 부산 동래구:1, 서울 용인구:1 -> 부산 금정구:1+2, 부산 동래구:1+2, 서울 용인구:1+1
def add_weight_by_region(region_dict):
    weight = {}
    for key in region_dict.keys():
        key = key.split(' ')[0]
        weight[key] = 1 + weight.get(key, 0)
    for key in region_dict.keys():
        region_dict[key] += weight[key.split(' ')[0]]
    return region_dict

In [12]:
def find_region(store_id):
    #해당 매장에서 결제된 고객들의 카드가 사용된 지역
    region_dict = check_store_region(store_id)
    #지역이 존재하면
    if len(region_dict)>0:
        #대도시 기준 가중치 부여
        region_dict = add_weight_by_region(region_dict)
        #지역과 최대값을 찾는다
        max_region = max(region_dict.items(), key=operator.itemgetter(1))
        #같은 값을 가지는 오차가 존재 할 가능성이 있으나, 앞의 가중치 부여로 어느정도 보정을 하였기 때문에
        #따로 수정하지는 않는다.
        return max_region[0]
    return np.nan

In [13]:
len(df[~df['region'].isnull()]['store_id'].unique()), len(df['store_id'].unique())
#총 1967 데이터 중 1277개 데이터가 비어있다.

(1277, 1967)

In [14]:
region_null_stores = df[df['region'].isnull()]['store_id'].unique()
for step, store_id in enumerate(region_null_stores):
    val = find_region(store_id)
    df.loc[df['store_id'] == store_id, 'region'] = val
    print("[*]{:>6}% completed".format((100*step/len(region_null_stores))), end="\r", flush=True)

In [15]:
df.isnull().sum()

store_id                  0
card_id                   0
card_company              0
transacted_date           0
transacted_time           0
installment_term          0
region                 8321
type_of_business    3952609
amount                    0
transacted_year           0
transacted_month          0
transacted_hour           0
type_big            3952609
dtype: int64

In [16]:
#데이터를 순회하며 지역의 결측값을 채울 때, 데이터가 부족하면 None으로 채워진다.
#이럴 경우, 해당 가게에서 사용된 card_id가 다른 매장에서 사용된 경우가 없는 경우이다.
#이를 확인하는 코드.
for store_id in df[df['region'].isnull()].store_id.unique():
    print('>>store_id : {}'.format(store_id))
    card_id_list = df[df['store_id']==store_id]['card_id'].unique()
    for card_id in card_id_list:
        unique_store_id = df[df['card_id']==card_id]['store_id'].unique()
        if len(unique_store_id) is not 1:
            print('!!{} - {} - {}'.format(store_id, card_id, unique_store_id))

>>store_id : 95
!!95 - 234158 - [  95 1185]
!!95 - 234267 - [  95 1513]
>>store_id : 125
!!125 - 297417 - [ 125 1950]
!!125 - 297500 - [125 557]
>>store_id : 314
>>store_id : 481
>>store_id : 564
>>store_id : 952
>>store_id : 1351
>>store_id : 1365


In [17]:
#새로 채워넣은 데이터를 바탕으로 한번 더 region의 결측값을 채운다
region_null_stores_2nd = df[df['region'].isnull()]['store_id'].unique()
for step, store_id in enumerate(region_null_stores_2nd):
    val = find_region(store_id)
    df.loc[df['store_id'] == store_id, 'region'] = val
    print("[*]{:.2} % completed".format(100*step/len(region_null_stores_2nd)), end="\r", flush=True)

In [18]:
#여기서 나오는 store_id에서 사용된 card_id는 다른 곳에서 사용된 기록이 없다.
#이 특징들을 살펴보자.
for store_id in df[df['region'].isnull()].store_id.unique():
    print('>>store_id : {}'.format(store_id))
    card_id_list = df[df['store_id']==store_id]['card_id'].unique()
    for card_id in card_id_list:
        unique_store_id = df[df['card_id']==card_id]['store_id'].unique()
        if len(unique_store_id) is not 1:
            print('!!{} - {} - {}'.format(store_id, card_id, unique_store_id))

>>store_id : 314
>>store_id : 481
>>store_id : 564
>>store_id : 952
>>store_id : 1351
>>store_id : 1365


In [19]:
df.isnull().sum()

store_id                  0
card_id                   0
card_company              0
transacted_date           0
transacted_time           0
installment_term          0
region                 6433
type_of_business    3952609
amount                    0
transacted_year           0
transacted_month          0
transacted_hour           0
type_big            3952609
dtype: int64

In [20]:
#해당 데이터는 region, type_of_business가 모두 결측값
df[df['store_id'].isin(df[df['region'].isnull()].store_id.unique())].groupby(['store_id']).agg({'region':'count','type_of_business':'count'})

,region,type_of_business
store_id,,
314,0,0
481,0,0
564,0,0
952,0,0
1351,0,0
1365,0,0


In [21]:
df['region'] = df['region'].fillna('기타 기타')

In [22]:
df['region_big'] = df['region'].apply(split_data, args=[0, ' '])
df['region_detail'] = df['region'].apply(split_data, args=[1, ' '])

In [23]:
df.isnull().sum()

store_id                  0
card_id                   0
card_company              0
transacted_date           0
transacted_time           0
installment_term          0
region                    0
type_of_business    3952609
amount                    0
transacted_year           0
transacted_month          0
transacted_hour           0
type_big            3952609
region_big                0
region_detail             0
dtype: int64

In [24]:
df.to_csv('df_fixed.csv')

In [25]:
print('finish')

finish
